In [1]:
%reset -f
%load_ext autoreload
%autoreload 2
import pandas as pd
import numpy as np
from linearmodels.iv import IV2SLS
import functions as fun
import statsmodels.formula.api as sm
import seaborn as sns
from matplotlib import pyplot as plt

In [2]:
df = pd.read_csv('dataset.csv')
Nobs=df['ID'].count()
df['const']=np.ones((Nobs,1))
df2 = df[df['Market share'] != 0]

df2.head(20)

,ID,Year,Market share,Manufacturer,Model,Range,Price,HP,Chargetime,Type,Segment,Country,Sales,const
8,1,2021,0.010373,Aiways,U5,400,284621.700,201,34,SUV,C,CN,257,1.0
9,1,2022,0.005976,Aiways,U5,400,313681.829,201,34,SUV,C,CN,183,1.0
10,1,2023,0.002860,Aiways,U5,400,264524.000,201,34,SUV,C,CN,177,1.0
21,2,2023,0.000048,Aiways,U6,405,360638.000,214,34,SUV,C,CN,3,1.0
28,3,2019,0.040630,Audi,e-tron,375,979704.475,402,17,SUV,F,DE,222,1.0
29,3,2020,0.034680,Audi,e-tron,375,890101.410,402,17,SUV,F,DE,491,1.0
30,3,2021,0.010494,Audi,e-tron,375,800035.193,402,17,SUV,F,DE,260,1.0
31,3,2022,0.017570,Audi,e-tron,375,789723.656,402,17,SUV,F,DE,538,1.0
32,3,2023,0.001099,Audi,e-tron,375,673037.728,402,17,SUV,F,DE,68,1.0
41,4,2021,0.003391,Audi,e-tron GT,472,1278896.110,522,17,Sedan,F,DE,84,1.0


In [3]:
# Copy the dataframe
data = df2.copy().reset_index(drop=True)
data.head(20)

,ID,Year,Market share,Manufacturer,Model,Range,Price,HP,Chargetime,Type,Segment,Country,Sales,const
0,1,2021,0.010373,Aiways,U5,400,284621.700,201,34,SUV,C,CN,257,1.0
1,1,2022,0.005976,Aiways,U5,400,313681.829,201,34,SUV,C,CN,183,1.0
2,1,2023,0.002860,Aiways,U5,400,264524.000,201,34,SUV,C,CN,177,1.0
3,2,2023,0.000048,Aiways,U6,405,360638.000,214,34,SUV,C,CN,3,1.0
4,3,2019,0.040630,Audi,e-tron,375,979704.475,402,17,SUV,F,DE,222,1.0
5,3,2020,0.034680,Audi,e-tron,375,890101.410,402,17,SUV,F,DE,491,1.0
6,3,2021,0.010494,Audi,e-tron,375,800035.193,402,17,SUV,F,DE,260,1.0
7,3,2022,0.017570,Audi,e-tron,375,789723.656,402,17,SUV,F,DE,538,1.0
8,3,2023,0.001099,Audi,e-tron,375,673037.728,402,17,SUV,F,DE,68,1.0
9,4,2021,0.003391,Audi,e-tron GT,472,1278896.110,522,17,Sedan,F,DE,84,1.0


In [4]:
#Scale for better intepretation

data['Price'] = data['Price']/10_000 #(Change in ms(%) for change in pris in 10.000)
data['HP'] = data['HP']/10           #(Change in ms(%) for change in HP in 10)
data['Range'] = data['Range']/10     #(Change in ms(%) for change in rækkevidde in 10)

In [5]:
# Creating dummy for china
data['China'] = (data['Country'] == 'CN').astype(int)

In [6]:
# Take the log of the market share
data['log_market_share'] = np.log(data['Market share'])

# OLS

In [7]:
OLS = sm.ols('log_market_share ~ Price + Range + HP + Chargetime + China', data).fit(cov_type='HC3')
OLS.summary()#.tables[1]

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:       log_market_share   R-squared:                       0.207
Model:                            OLS   Adj. R-squared:                  0.195
Method:                 Least Squares   F-statistic:                     21.01
Date:                Thu, 09 May 2024   Prob (F-statistic):           3.18e-18
Time:                        12:17:32   Log-Likelihood:                -716.37
No. Observations:                 334   AIC:                             1445.
Df Residuals:                     328   BIC:                             1468.
Df Model:                           5                                         
Covariance Type:                  HC3                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -6.4240      0.700     -9.171      0.000      -7.797      -5.051
Price         -0.0322      0.005     -6.537      0.000      -0.042      -0.023
Range          0.0835      0.014      5.836      0.000       0.055       0.112
HP             0.0017      0.012      0.136      0.892      -0.022       0.026
Chargetime    -0.0132      0.012     -1.058      0.290      -0.038       0.011
China         -2.2275      0.362     -6.152      0.000      -2.937      -1.518
==============================================================================
Omnibus:                       16.332   Durbin-Watson:                   0.928
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               17.595
Skew:                          -0.560   Prob(JB):                     0.000151
Kurtosis:                       3.096   Cond. No.                         450.
==============================================================================

Notes:
[1] Standard Errors are heteroscedasticity robust (HC3)
"""

# Willingness to pay

In [8]:
alpha = OLS.params[1] # Price coefficient
beta = list(OLS.params[:1]) + list(OLS.params[2:]) # Other coefficients
beta_alpha_ratio = [b / -alpha for b in beta[1:]] #Willingness to pay (excluding constant)

for i in range(len(OLS.params.index[2:])): #[2:] to exclude constant and price
    print('W2P:', OLS.params.index[2:][i], beta_alpha_ratio[i])

W2P: Range 2.592680339023965
W2P: HP 0.05154449225980682
W2P: Chargetime -0.40934384664303847
W2P: China -69.14201739405368


25,926 DKK for every 10 increase in Range\
515 DKK for every 10 increase in HP\
-4,093 DKK for every 10 increase in Chargetime\
-691,420 DKK for Chinese cars OR -69,000 DKK for Chinese cars???

# Logit

In [9]:
intercept = np.ones((len(data),1))
data['Intercept'] = intercept

In [10]:
X = data[['Intercept', 'Range', 'HP', 'Chargetime', 'China']]
p_j = data['Price']

In [11]:
# IMPORTANT: The data must be sorted by year and ID before running the function
#             Because the CCPs returned by the function are sorted by year and ID
logit_data = data.sort_values(['Year', 'ID']).reset_index(drop=True)
logit_data['CCP'] = fun.ccp(alpha, beta, data, X)

In [12]:
logit_data['Model_year'] = logit_data['Model'] + '_' + logit_data['Year'].astype(str)
probability_ratio = fun.probability_ratio(logit_data['CCP'], logit_data['Model_year'])
probability_ratio

Model_year,I3_2013,Leaf_2013,Zoe_2013,Fortwo_2013,Model S_2013,up!_2013,I3_2014,Leaf_2014,Zoe_2014,Model S_2014,...,ID.3_2023,ID.4_2023,ID.5_2023,ID.7_2023,ID.Buzz_2023,up!_2023,C40_2023,EX30_2023,XC40_2023,Free_2023
Model_year,,,,,,,,,,,,,,,,,,,,,
I3_2013,1.0,1.071713,0.722054,6.322656,0.35485,1.632381,1.223077,1.178589,0.77673,0.32625,...,9.299983,6.224527,5.50374,3.573844,19.226011,29.662927,8.524486,6.612304,9.448216,106.47518
Leaf_2013,0.933086,1.0,0.673739,5.899579,0.331105,1.523151,1.141236,1.099724,0.724755,0.304419,...,8.67768,5.808016,5.13546,3.334702,17.939512,27.678048,7.954075,6.169845,8.815994,99.35045
Zoe_2013,1.384937,1.484255,1.0,8.756481,0.491445,2.260745,1.693885,1.632272,1.075722,0.451836,...,12.879891,8.620578,7.622333,4.949549,26.626815,41.081287,11.805877,9.157625,13.085185,147.461424
Fortwo_2013,0.158161,0.169504,0.114201,1.0,0.056124,0.25818,0.193444,0.186407,0.122849,0.0516,...,1.470898,0.98448,0.870479,0.565244,3.040812,4.691529,1.348244,1.045811,1.494343,16.840261
Model S_2013,2.818092,3.020186,2.034816,17.817827,1.0,4.600199,3.446744,3.321372,2.188896,0.919403,...,26.208208,17.54129,15.510045,10.071421,54.180667,83.592856,24.022787,18.634081,26.625943,300.056853
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
up!_2023,0.033712,0.03613,0.024342,0.21315,0.011963,0.055031,0.041233,0.039733,0.026185,0.010999,...,0.313522,0.209842,0.185543,0.120482,0.648149,1.0,0.287378,0.222915,0.318519,3.589504
C40_2023,0.117309,0.125722,0.084704,0.741705,0.041627,0.191493,0.143478,0.138259,0.091117,0.038272,...,1.090973,0.730194,0.645639,0.419245,2.255386,3.479732,1.0,0.775684,1.108362,12.49051
EX30_2023,0.151233,0.162079,0.109199,0.956196,0.053665,0.24687,0.18497,0.178242,0.117467,0.04934,...,1.406466,0.941355,0.832348,0.540484,2.907611,4.48602,1.289186,1.0,1.428884,16.102584


# Det ligner at BLP har constant termet med i marginal effect; se Table IV

In [13]:
marginal_effects = fun.marginal_effects(logit_data['CCP'], logit_data['Model_year'], OLS.params.index, OLS.params)
marginal_effects

,Intercept,Price,Range,HP,Chargetime,China
Model_year,,,,,,
I3_2013,-0.795428,-0.003989,0.010342,0.000206,-0.001633,-0.275805
Leaf_2013,-0.75061,-0.003764,0.009759,0.000194,-0.001541,-0.260265
Zoe_2013,-1.029828,-0.005164,0.01339,0.000266,-0.002114,-0.357081
Fortwo_2013,-0.143736,-0.000721,0.001869,0.000037,-0.000295,-0.049839
Model S_2013,-1.551645,-0.007781,0.020174,0.000401,-0.003185,-0.538015
...,...,...,...,...,...,...
up!_2023,-0.031202,-0.000156,0.000406,0.000008,-0.000064,-0.010819
C40_2023,-0.107255,-0.000538,0.001395,0.000028,-0.00022,-0.037189
EX30_2023,-0.13758,-0.00069,0.001789,0.000036,-0.000282,-0.047704


In [14]:
marginal_effects[OLS.params.index].mean()

Intercept    -0.184726
Price        -0.000926
Range         0.002402
HP            0.000048
Chargetime   -0.000379
China        -0.064051
dtype: float64

In [17]:
elasticity = fun.elasticity(logit_data['CCP'], logit_data['Model_year'], OLS.params.index, OLS.params, logit_data[OLS.params.index])
elasticity

elasticity shape: 
(334, 6)


,Intercept,Price,Range,HP,Chargetime,China
Model_year,,,,,,
I3_2013,[-5.493922806426262],[-0.6887832825851413],[2.1072379520361224],[0.023715993698033296],[-0.2030034228457109],[-0.0]
Leaf_2013,[-5.556159223014567],[-0.696585977458081],[2.131109226470522],[0.023984653909616523],[-0.20530309214181008],[-0.0]
Zoe_2013,[-5.135897043443153],[-0.6438969292513987],[1.9699143124170475],[0.022170479310989286],[-0.18977417702021374],[-0.0]
Fortwo_2013,[-6.27690770935364],[-0.7869475507510679],[2.4075580623569217],[0.027095958374959653],[-0.2319351390221471],[-0.0]
Model S_2013,[-3.8029353149712097],[-0.4767810473494291],[1.4586462000289002],[0.016416391919156458],[-0.1405205193085286],[-0.0]
...,...,...,...,...,...,...
up!_2023,[-6.3926566028067695],[-0.8014592040081884],[2.4519544744989554],[0.02759561956868407],[-0.23621212331724925],[-0.0]
C40_2023,[-6.3149039430058895],[-0.7917112089717941],[2.422131821109556],[0.027259979324943097],[-0.23333912043186578],[-0.0]
EX30_2023,[-6.28335151181292],[-0.7877554222058762],[2.410029634232983],[0.02712377477935778],[-0.23217324101255823],[-0.0]


In [16]:
cross_elasticity = fun.cross_elasticity(logit_data['CCP'], beta, X, logit_data['Model'])
cross_elasticity

InvalidIndexError: (slice(None, None, None), slice(1, None, None))